# 🧠 Week 2: From Graphs to Real Agents

## LangGraph Deep Dive — Stateful Multi-Agent Systems

Welcome to Week 2! Last week you drew graphs, built nodes, and wired edges. This week, your graphs learn to **think, act, and remember**.

> **📚 Official Documentation:** This notebook follows the [LangGraph Official Documentation](https://docs.langchain.com/oss/python/langgraph/overview). All APIs and patterns used here are based on the current LangGraph specification.

---

## 🚀 What You'll Learn Today

1. **Typed State** — Design state like an engineer with proper types and reducers
2. **ReAct Pattern** — The core loop that turns graphs into agents (Reason → Act → Observe → Repeat)
3. **Conditional Edges & Termination** — How agents decide what happens next
4. **Checkpointing** — Give your agents memory and replay capabilities
5. **Live Build** — Build a Meeting Prep Agent from scratch

### 🎯 By the End of This Notebook

- ✅ Understand typed state and reducers
- ✅ Implement the ReAct pattern in LangGraph
- ✅ Build agents with memory using checkpointing
- ✅ Create a working Meeting Prep Agent
- ✅ Be ready for multi-agent systems in Week 3

---

### 📖 How to Use This Notebook

1. **Run cells in order** - Each cell builds on the previous one
2. **Read the markdown cells** - They contain important explanations
3. **Experiment** - Try modifying the code to see what happens
4. **Complete the challenge** - Build your own extended agent

**Ready? Let's start by installing dependencies!**


## 🔧 Setup & Installation


In [ ]:
# Install dependencies (run once)
# If you have requirements.txt in the same directory, you can use:
# %pip install -r requirements.txt

# Otherwise, install individually:
%pip install langgraph langchain-core langchain-openai langchain ipython python-dotenv tavily-python

# Verify installation
import sys
print(f"Python version: {sys.version}")
print("✅ Dependencies installed successfully!")
print("💡 Make sure you have a .env file with OPENAI_API_KEY for the agent to work!")


---

## 📦 Imports

Let's import all the essential components we'll need:


In [ ]:
# Core imports
import operator
from typing import Annotated, Literal
from typing_extensions import TypedDict
import os
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# LangChain imports
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# Tavily Search import
try:
    from tavily import TavilyClient
    TAVILY_AVAILABLE = True
except ImportError:
    TAVILY_AVAILABLE = False
    print("⚠️  Tavily not installed. Install with: pip install tavily-python")

# Verify API keys are loaded
api_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

if api_key:
    print("✅ OpenAI API key loaded from .env")
else:
    print("⚠️  Warning: OPENAI_API_KEY not found in .env file")

if tavily_key and TAVILY_AVAILABLE:
    print("✅ Tavily API key loaded from .env")
    tavily_client = TavilyClient(api_key=tavily_key)
else:
    print("⚠️  Warning: TAVILY_API_KEY not found or Tavily not installed")
    tavily_client = None

print("✅ All imports successful!")
print("📚 Using official LangGraph API from: https://docs.langchain.com/oss/python/langgraph/overview")


---

# 📓 Concept 1: Typed State

## Why State Design Matters

In Week 1, you learned about TypedDict and basic state design. This week, we're going **deeper** into state design for production agents with multiple fields, different reducer patterns, and more complex state management.

### Building on Week 1

Week 1 introduced:
- `TypedDict` for state structure
- `Annotated[list, operator.add]` for append-only fields
- Basic state management

Week 2 expands this with:
- **Multiple state fields** with different update patterns
- **ReAct trace tracking** (`steps` field)
- **Final answer handling** (overwrite vs append)
- **More complex state schemas** for real agents

### The Solution: TypedDict + Reducers

**TypedDict** defines the shape of your state.
**Annotated + Reducers** control how updates are merged.

### The Rule

Nodes either **append** to `messages`/`steps` OR **set** `final_answer`. Never both.


In [ ]:
# Define a proper typed state for an agent
class AgentState(TypedDict):
    # Append-only fields (use operator.add reducer)
    messages: Annotated[list, operator.add]  # Conversation history
    steps: Annotated[list, operator.add]     # ReAct trace: action + observation pairs
    
    # Overwrite fields (no reducer)
    final_answer: str  # Completed output (set once)

print("✅ AgentState defined!")


### Understanding Reducers

**Reducers** determine how state updates are merged:

| Field | Reducer | Behavior |
|-------|---------|----------|
| `messages` | `operator.add` | **Append** new messages to existing list |
| `steps` | `operator.add` | **Append** new steps to existing list |
| `final_answer` | None | **Overwrite** the field completely |

### Why This Matters

```python
# Without reducer (BAD):
# Node 1 returns: {"steps": [step_1]}
# State: [step_1]
# Node 2 returns: {"steps": [step_2]}
# State: [step_2] ← step_1 GONE! 😱

# With operator.add (GOOD):
# Node 1 returns: {"steps": [step_1]}
# State: [step_1]
# Node 2 returns: {"steps": [step_2]}
# State: [step_1, step_2] ✓
```

**Common gotcha:** If you forget the reducer, your second node erases everything the first node wrote. This is the #1 LangGraph debugging headache!


### Understanding State Updates in Practice

The key difference is how state updates are merged. Let's see this in action with a simple example:


In [ ]:
# Quick demonstration: Why reducers matter
# This shows the difference between append (with reducer) and overwrite (without reducer)

# Simulate state updates from two nodes
state = {
    "messages": [],
    "steps": [],
    "final_answer": ""
}

# Node 1 returns an update
node1_update = {"messages": [AIMessage("Hello!")], "steps": [{"type": "reason", "content": "Thinking..."}]}
# With reducer: messages get appended
state["messages"] = operator.add(state["messages"], node1_update["messages"])
state["steps"] = operator.add(state["steps"], node1_update["steps"])
print(f"After Node 1:")
print(f"  Messages: {len(state['messages'])} (content: {[m.content for m in state['messages']]})")
print(f"  Steps: {len(state['steps'])}")

# Node 2 returns another update
node2_update = {"messages": [AIMessage("How can I help?")], "steps": [{"type": "act", "tool": "lookup"}]}
# With reducer: messages get appended (not replaced!)
state["messages"] = operator.add(state["messages"], node2_update["messages"])
state["steps"] = operator.add(state["steps"], node2_update["steps"])
print(f"\nAfter Node 2:")
print(f"  Messages: {len(state['messages'])} (content: {[m.content for m in state['messages']]})")
print(f"  Steps: {len(state['steps'])}")
print(f"  ✅ Both nodes' updates preserved!")

# Final answer: overwrite (no reducer)
state["final_answer"] = "Original First answer"
state["final_answer"] = "Final answer"  # Overwrites previous
print(f"\nFinal answer: {state['final_answer']}")
print(f"  ✅ Overwrites previous value (no reducer)")


---

# 🔄 Concept 2: The ReAct Pattern

## Reason → Act → Observe → Repeat

This is the **core loop** that turns a graph into an agent.

### The ReAct Flow

```
    Think (Reasoner Node)
         ↓
    Act (Tool Node)
         ↓
    Observe (Tool Result)
         ↓
    Repeat or Done
```

### Key Components

1. **Reasoner Node** — LLM looks at state, decides what to do next
   - "I need more context" → call a tool
   - "I have enough" → produce final answer

2. **Tool Node** — Executes the tool, returns observation

3. **Conditional Router** — Decides whether to continue or finish

**The LLM doesn't just answer. It *thinks* about what it needs, *acts* to get it, *observes* the result, and *decides again*.**


### Step 1: Define Tools

Tools are functions the agent can call to get information or perform actions.


In [ ]:
# Define REAL tools for our agent

@tool
def tavily_search(query: str) -> str:
    """Search the web for current information using Tavily API.
    
    This is a REAL web search tool that demonstrates the ReAct architecture.
    The agent can use this to gather up-to-date information from the internet.
    
    Args:
        query: The search query to look up on the web
    
    Returns:
        A formatted summary of search results with relevant information
    """
    if not tavily_client:
        return "Tavily API not available. Please install tavily-python (`pip install tavily-python`) and set TAVILY_API_KEY in .env file."
    
    try:
        # Perform real web search using Tavily
        response = tavily_client.search(
            query=query,
            max_results=5,
            search_depth="advanced"
        )
        
        # Format the results
        results = []
        if response.get('results'):
            for result in response['results']:
                title = result.get('title', 'No title')
                url = result.get('url', '')
                content = result.get('content', '')
                results.append(f"**{title}**\n{content[:200]}...\nSource: {url}")
        
        if results:
            return "\n\n".join(results)
        else:
            return f"No results found for query: {query}"
    except Exception as e:
        return f"Error searching: {str(e)}"


@tool
def get_todays_events() -> str:
    """Get today's calendar events from the calendar.
    
    Returns a list of today's scheduled meetings and events.
    This is a dummy calendar implementation for demonstration.
    
    Returns:
        A formatted list of today's calendar events
    """
    # Dummy calendar with sample events
    today = datetime.now().strftime("%Y-%m-%d")
    events = [
        {
            "time": "10:00 AM",
            "title": "Team Standup",
            "attendees": ["Team"],
            "location": "Zoom"
        },
        {
            "time": "2:00 PM",
            "title": "Meeting with Marc Kligen about Langfuse Eval",
            "attendees": ["Marc Kligen (Langfuse)", "You"],
            "location": "Google Meet",
            "topic": "Langfuse Eval features and integration"
        },
        {
            "time": "4:30 PM",
            "title": "Product Review",
            "attendees": ["Product Team"],
            "location": "Conference Room A"
        }
    ]
    
    formatted_events = [f"📅 {today} - Today's Events:\n"]
    for event in events:
        formatted_events.append(
            f"  ⏰ {event['time']}: {event['title']}\n"
            f"     👥 Attendees: {', '.join(event['attendees'])}\n"
            f"     📍 Location: {event['location']}"
        )
        if 'topic' in event:
            formatted_events.append(f"     💬 Topic: {event['topic']}")
        formatted_events.append("")
    
    return "\n".join(formatted_events)


@tool
def get_current_date() -> str:
    """Returns today's date and time for context.
    
    Returns:
        Current date and time in a readable format
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


# Create a list of tools
tools = [tavily_search, get_todays_events, get_current_date]

print(f"✅ Defined {len(tools)} tools:")
for t in tools:
    print(f"  - {t.name}")
    if t.name == "tavily_search" and tavily_client:
        print("    🔍 Real web search API - demonstrates ReAct architecture!")
    elif t.name == "get_todays_events":
        print("    📅 Calendar tool - shows today's meetings")


### Step 2: Create the LLM with Tools

We'll use OpenAI's ChatOpenAI model, bound with our tools so it can call them.


In [ ]:
# Create the LLM with tools bound
# The API key is loaded from .env file in the imports cell above

if api_key:
    # Create the LLM instance
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # Bind tools to the LLM so it can call them
    llm_with_tools = llm.bind_tools(tools)
    
    print("✅ LLM initialized with tools")
    print(f"📝 Model: gpt-4o-mini")
    print(f"🔧 Tools available: {[tool.name for tool in tools]}")
else:
    llm = None
    llm_with_tools = None
    print("⚠️  Warning: Cannot initialize LLM without API key")


### Step 3: Define the ReAct Nodes

We need two main nodes:
1. **Reasoner Node** — Calls the LLM to decide what to do
2. **Tool Node** — Executes the tool and returns the observation


In [ ]:
# Reasoner node using actual LLM
def reasoner_node(state: AgentState) -> dict:
    """Reasoner node: LLM decides what to do next.
    
    This is the 'Think' step of the ReAct loop.
    The LLM examines the current state and either:
    - Calls a tool (needs more info)
    - Returns a final answer (has enough info)
    """
    messages = state.get("messages", [])
    
    if not messages:
        return {"messages": [AIMessage("I need a question to help with!")]}
    
    last_message = messages[-1]
    
    # If we already have a final answer, we're done
    if state.get("final_answer"):
        return {}
    
    '''
    # ── Fallback if no LLM configured ──
    if not llm_with_tools:
        if isinstance(last_message, HumanMessage):
            content = last_message.content.lower()
            if "marc" in content or "langfuse" in content or "eval" in content:
                tool_call_message = AIMessage(
                    content="",
                    tool_calls=[{
                        "name": "tavily_search",
                        "args": {"query": "Langfuse Eval Marc Kligen"},
                        "id": "call_123"
                    }]
                )
                return {
                    "messages": [tool_call_message],
                    "steps": [{"type": "reason", "content": "I need to research Langfuse Eval and Marc Kligen."}]
                }
            else:
                final = "I can help you prepare. Let me gather context first."
                return {"messages": [AIMessage(final)], "final_answer": final}
        elif isinstance(last_message, ToolMessage):
            tool_result = last_message.content
            final = f"Based on research:\n{tool_result[:300]}\n\nMeeting prep complete!"
            return {
                "messages": [AIMessage(final)],
                "steps": [{"type": "reason", "content": "Generating final meeting prep summary."}],
                "final_answer": final
            }
        return {}
    '''
    # ── Use actual LLM ──
    try:
        if isinstance(last_message, ToolMessage):
            # After tool execution: synthesize a final answer (no tools bound)
            # Build clean message history to avoid tool_call_id mismatch errors
            clean_messages = []
            tool_msgs = []
            ai_with_tools = None
            human_msg = None
            
            for msg in reversed(messages):
                if isinstance(msg, ToolMessage):
                    tool_msgs.insert(0, msg)
                elif isinstance(msg, AIMessage) and getattr(msg, 'tool_calls', None):
                    ai_with_tools = msg
                    break
                elif isinstance(msg, HumanMessage) and human_msg is None:
                    human_msg = msg
            
            if human_msg and ai_with_tools and tool_msgs:
                clean_messages = [human_msg, ai_with_tools] + tool_msgs
            else:
                clean_messages = messages[-3:] if len(messages) >= 3 else messages
            
            try:
                response = llm.invoke(clean_messages)          # plain llm, no tools
                final_answer = response.content
                return {
                    "messages": [response],
                    "steps": [{"type": "reason", "content": "Synthesizing final answer from tool results."}],
                    "final_answer": final_answer
                }
            except Exception as e:
                print(f"⚠️  Error calling LLM: {e}")
                final_answer = f"Based on tool results: {last_message.content}"
                return {
                    "messages": [AIMessage(final_answer)],
                    "steps": [{"type": "reason", "content": "Generated answer from tool results."}],
                    "final_answer": final_answer
                }
        else:
            # First reasoning step: ask LLM (with tools) what to do
            response = llm_with_tools.invoke(messages)
            
            if hasattr(response, 'tool_calls') and response.tool_calls:
                return {
                    "messages": [response],
                    "steps": [{"type": "reason",
                               "content": response.content or "Deciding to call a tool to gather more information."}]
                }
            else:
                final_answer = response.content
                return {
                    "messages": [response],
                    "steps": [{"type": "reason", "content": "Generated final answer."}],
                    "final_answer": final_answer
                }
    except Exception as e:
        print(f"⚠️  Error calling LLM: {e}")
        return {
            "messages": [AIMessage(f"Error: {str(e)}")],
            "final_answer": f"Error: {str(e)}"
        }


def tool_node(state: AgentState) -> dict:
    """Tool node: Executes tool calls and returns observations.
    
    This is the 'Act' step of the ReAct loop.
    It finds the last AIMessage with tool_calls, executes each,
    and returns ToolMessage results.
    """
    messages = state.get("messages", [])
    last_message = messages[-1] if messages else None
    
    if not last_message or not getattr(last_message, 'tool_calls', None):
        return {}
    
    tool_map = {t.name: t for t in tools}
    tool_messages = []
    tool_steps = []
    
    for tc in last_message.tool_calls:
        name = tc["name"]
        args = tc["args"]
        call_id = tc["id"]
        
        if name in tool_map:
            try:
                result = tool_map[name].invoke(args)
                tool_messages.append(ToolMessage(content=str(result), tool_call_id=call_id))
                tool_steps.append({"type": "act", "tool": name, "args": args, "result": str(result)[:200]})
            except Exception as e:
                tool_messages.append(ToolMessage(content=f"Error: {e}", tool_call_id=call_id))
        else:
            tool_messages.append(ToolMessage(content=f"Unknown tool: {name}", tool_call_id=call_id))
    
    return {"messages": tool_messages, "steps": tool_steps} if tool_messages else {}


print("✅ ReAct nodes defined: reasoner_node, tool_node")


---

# 🎯 Concept 3: Conditional Edges & Termination

## The Router: How Your Graph Decides What Happens Next

**Agents are just graphs with loops + a stopping policy.**

### The Routing Function

The router examines the state and decides:
- If LLM requested a tool → route to `tool_node`
- If LLM said DONE → route to `final_node`
- If parsing failed → route to `retry_node` (optional)

### Critical: Add a Max-Steps Guard

**Always** add a maximum steps limit to prevent infinite loops!


In [ ]:
# Router: decides whether to continue or finish
MAX_STEPS = 10  # Safety guard against infinite loops

def should_continue(state: AgentState) -> Literal["tool_node", "__end__"]:
    """Conditional edge: route to tool_node or END.
    
    Checks:
    1. If we have a final answer → END
    2. If the last AI message has tool_calls → tool_node
    3. If we've exceeded MAX_STEPS → END (safety)
    4. Otherwise → END
    """
    # Safety: max steps guard
    if len(state.get("steps", [])) >= MAX_STEPS:
        print(f"⚠️  Max steps ({MAX_STEPS}) reached — terminating")
        return "__end__"
    
    # If final answer is set, we're done
    if state.get("final_answer"):
        return "__end__"
    
    # Check last message
    messages = state.get("messages", [])
    if messages:
        last = messages[-1]
        if isinstance(last, AIMessage) and getattr(last, 'tool_calls', None):
            return "tool_node"
    
    return "__end__"


print("✅ Router defined: should_continue()")
print(f"   Max steps guard: {MAX_STEPS}")


### Building the ReAct Graph

Now let's wire everything together:


In [ ]:
# Build the ReAct graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("reasoner", reasoner_node)
workflow.add_node("tool_node", tool_node)

# Add edges
workflow.add_edge(START, "reasoner")                     # START → reasoner
workflow.add_conditional_edges("reasoner", should_continue)  # reasoner → tool_node | END
workflow.add_edge("tool_node", "reasoner")               # tool_node → reasoner (loop back)

# Compile the graph (no checkpointer yet)
react_app = workflow.compile()

print("✅ ReAct graph compiled!")
print("   START → reasoner ─┬─→ tool_node → reasoner (loop)")
print("                     └─→ END")


### Visualize the ReAct Graph


In [ ]:
# Visualize the ReAct graph
try:
    from IPython.display import Image, display
    display(Image(react_app.get_graph().draw_mermaid_png()))
except Exception as e:
    # Fallback: print the mermaid diagram
    print("Graph structure (Mermaid):")
    print(react_app.get_graph().draw_mermaid())


### Test the ReAct Agent

Let's test our agent with a simple query:


In [ ]:
# Test the ReAct agent with a simple query
print("=" * 60)
print("REACT AGENT TEST")
print("=" * 60)

initial_state = {
    "messages": [HumanMessage("What is the current date and time?")],
    "steps": [],
    "final_answer": ""
}

result = react_app.invoke(initial_state)

# Display the trace
print("\n📋 REACT TRACE:")
for i, step in enumerate(result.get("steps", []), 1):
    step_type = step.get("type", "unknown")
    if step_type == "reason":
        print(f"  Step {i} - 💭 REASON: {step.get('content', '')[:100]}")
    elif step_type == "act":
        print(f"  Step {i} - 🔧 ACT: {step.get('tool', '')}({step.get('args', {})})")
        print(f"           📊 Result: {step.get('result', '')[:100]}")

print("\n" + "=" * 60)
print("FINAL ANSWER:")
print("=" * 60)
print(result.get("final_answer", "No answer generated"))


---

## 🎯 ReAct Examples: Real-World Patterns

Now let's see the ReAct pattern in action with **real tools**! These examples demonstrate different agent behaviors:

1. **Example 1: Simple Single Tool Call** - Agent uses one tool to answer a question
2. **Example 2: Multi-Step Reasoning** - Agent chains multiple tool calls to solve a complex task
3. **Example 3: Tool Selection** - Agent chooses the right tool from multiple options


### Example 1: Simple Single Tool Call

**Pattern:** User asks a question → Agent reasons → Agent calls ONE tool → Agent synthesizes answer

This is the simplest ReAct pattern. The agent makes a single tool call to gather information, then provides a final answer.


In [ ]:
# Example 1: Single Tool Call
# The agent searches the web to answer a factual question
print("=" * 60)
print("EXAMPLE 1: Simple Single Tool Call")
print("=" * 60)

state_ex1 = {
    "messages": [HumanMessage("What is Langfuse and what does it do?")],
    "steps": [],
    "final_answer": ""
}

result_ex1 = react_app.invoke(state_ex1)

# Show the ReAct trace
print("\n📋 REACT TRACE:")
for i, step in enumerate(result_ex1.get("steps", []), 1):
    step_type = step.get("type", "unknown")
    if step_type == "reason":
        print(f"\n  Step {i} - 💭 REASON:")
        print(f"    {step.get('content', '')[:150]}")
    elif step_type == "act":
        print(f"\n  Step {i} - 🔧 ACT: {step.get('tool', '')}()")
        result_preview = step.get('result', '')[:150]
        print(f"    📊 Result: {result_preview}...")

print("\n" + "=" * 60)
print("✅ FINAL ANSWER:")
print("=" * 60)
print(result_ex1.get("final_answer", "No answer")[:500])


### Example 2: Multi-Step Reasoning

**Pattern:** User asks a complex question → Agent reasons → Agent calls MULTIPLE tools in sequence → Agent synthesizes comprehensive answer

This demonstrates how agents can chain tool calls. The agent might first check the calendar, then search for additional context, then provide a comprehensive answer.


In [ ]:
# Example 2: Multi-Step Reasoning
# The agent checks the calendar FIRST, then searches for context
print("=" * 60)
print("EXAMPLE 2: Multi-Step Reasoning")
print("=" * 60)

state_ex2 = {
    "messages": [HumanMessage(
        "Check my calendar for today and then research the topics "
        "for any meetings I have so I can be prepared."
    )],
    "steps": [],
    "final_answer": ""
}

result_ex2 = react_app.invoke(state_ex2)

# Show the ReAct trace
print("\n📋 REACT TRACE:")
for i, step in enumerate(result_ex2.get("steps", []), 1):
    step_type = step.get("type", "unknown")
    if step_type == "reason":
        print(f"\n  Step {i} - 💭 REASON:")
        print(f"    {step.get('content', '')[:150]}")
    elif step_type == "act":
        print(f"\n  Step {i} - 🔧 ACT: {step.get('tool', '')}()")
        result_preview = step.get('result', '')[:150]
        print(f"    📊 Result: {result_preview}...")

print(f"\n📊 Total steps: {len(result_ex2.get('steps', []))}")
print("\n" + "=" * 60)
print("✅ FINAL ANSWER:")
print("=" * 60)
print(result_ex2.get("final_answer", "No answer")[:800])


### Example 3: Tool Selection

**Pattern:** User asks a question → Agent reasons → Agent SELECTS the most appropriate tool → Agent provides answer

This shows how agents intelligently choose between available tools based on the query. The agent must decide whether to search the web, check the calendar, or get the current date.


In [ ]:
# Example 3: Tool Selection
# The agent must CHOOSE the right tool — no web search needed here
print("=" * 60)
print("EXAMPLE 3: Tool Selection")
print("=" * 60)

state_ex3 = {
    "messages": [HumanMessage("What meetings do I have today and what time is it now?")],
    "steps": [],
    "final_answer": ""
}

result_ex3 = react_app.invoke(state_ex3)

# Show the ReAct trace
print("\n📋 REACT TRACE:")
for i, step in enumerate(result_ex3.get("steps", []), 1):
    step_type = step.get("type", "unknown")
    if step_type == "reason":
        print(f"\n  Step {i} - 💭 REASON:")
        print(f"    {step.get('content', '')[:150]}")
    elif step_type == "act":
        print(f"\n  Step {i} - 🔧 ACT: {step.get('tool', '')}()")
        result_preview = step.get('result', '')[:150]
        print(f"    📊 Result: {result_preview}...")

print(f"\n📊 Total steps: {len(result_ex3.get('steps', []))}")
print("\n" + "=" * 60)
print("✅ FINAL ANSWER:")
print("=" * 60)
print(result_ex3.get("final_answer", "No answer")[:500])

print("\n" + "=" * 60)
print("🎯 KEY TAKEAWAY:")
print("The agent chose get_todays_events AND get_current_date")
print("without using tavily_search — because web search wasn't needed!")
print("=" * 60)


---

# 💾 Concept 4: Checkpointing

## Your Agent Gets Memory and Replay

**Checkpointing** saves the full state after every node execution. This enables:

| Feature | Description |
|---------|-------------|
| **Memory** | Same `thread_id` = agent remembers the entire conversation |
| **Fault Tolerance** | Crash mid-run? Resume from last checkpoint |
| **Time Travel** | Replay any past state, branch from any point |
| **Debugging** | Inspect exactly what the agent saw at each step |

### Without Checkpointing

Every invocation starts from scratch. No memory, no recovery.

### With Checkpointing

Full state saved after every node execution. Persistent across runs.

**Reference:** [LangGraph Official Documentation - Persistence](https://docs.langchain.com/oss/python/langgraph/overview)


### Setting Up Checkpointing

We'll use **MemorySaver** as our checkpoint store for in-memory persistence. This is the standard approach recommended in the [official LangGraph documentation](https://docs.langchain.com/oss/python/langgraph/overview).

**Note:** For production deployments, you can use other checkpoint stores (like Postgres, SQLite, or custom implementations), but MemorySaver is perfect for learning and development.


In [ ]:
# Set up checkpointing with MemorySaver
memory = MemorySaver()

# Recompile the graph WITH checkpointing enabled
react_app_with_memory = workflow.compile(checkpointer=memory)

print("✅ Graph recompiled with MemorySaver checkpointing!")
print("   Now the agent remembers conversations across invocations.")
print("   Use 'thread_id' in config to maintain conversation state.")


### Using Thread IDs for Memory

**Thread ID = Conversation Identity**

- Same `thread_id` → Agent remembers everything
- Different `thread_id` → Fresh start, clean state


In [ ]:
# Demonstrate memory with thread IDs
print("=" * 60)
print("CHECKPOINTING DEMO: Memory with Thread IDs")
print("=" * 60)

# Thread 1: First message
config_thread1 = {"configurable": {"thread_id": "demo-thread-1"}}

state1 = {
    "messages": [HumanMessage("Hi! My name is Alex and I'm working on a LangGraph project.")],
    "steps": [],
    "final_answer": ""
}

result1 = react_app_with_memory.invoke(state1, config=config_thread1)
print("\n🧵 Thread 1 — Message 1:")
print(f"   User: {state1['messages'][0].content}")
print(f"   Agent: {result1.get('final_answer', '')[:200]}")

# Thread 1: Second message — agent should REMEMBER the name
state2 = {
    "messages": [HumanMessage("What's my name and what am I working on?")],
    "steps": [],
    "final_answer": ""
}

result2 = react_app_with_memory.invoke(state2, config=config_thread1)
print(f"\n🧵 Thread 1 — Message 2 (same thread — should remember!):")
print(f"   User: {state2['messages'][0].content}")
print(f"   Agent: {result2.get('final_answer', '')[:200]}")

# Thread 2: Different thread — agent should NOT remember
config_thread2 = {"configurable": {"thread_id": "demo-thread-2"}}

state3 = {
    "messages": [HumanMessage("What's my name?")],
    "steps": [],
    "final_answer": ""
}

result3 = react_app_with_memory.invoke(state3, config=config_thread2)
print(f"\n🧵 Thread 2 — Different thread (should NOT remember!):")
print(f"   User: {state3['messages'][0].content}")
print(f"   Agent: {result3.get('final_answer', '')[:200]}")

print("\n" + "=" * 60)
print("🎯 KEY TAKEAWAY:")
print("   Same thread_id → Agent remembers everything")
print("   Different thread_id → Fresh start, no memory")
print("=" * 60)


---

# 🏗️ Live Build: Meeting Prep Agent

## Complete Implementation

Let's build a complete Meeting Prep Agent that:
- Takes a meeting request
- Uses tools to gather context
- Generates agenda bullets, questions, and demo ideas

**Prompt:** Prep me for a meeting with Marc Kligen about Langfuse Eval. Give me 5 agenda bullets, 3 questions to ask, and one demo idea.


In [ ]:
# Meeting Prep Agent — uses the same ReAct graph but with a rich prompt
# This demonstrates a real-world use case

class MeetingState(TypedDict):
    messages: Annotated[list, operator.add]
    steps: Annotated[list, operator.add]
    final_answer: str

def meeting_reasoner(state: MeetingState) -> dict:
    """Meeting prep reasoner: same pattern, richer prompt."""
    messages = state.get("messages", [])
    if not messages:
        return {}
    
    last_message = messages[-1]
    
    if state.get("final_answer"):
        return {}
    
    if not llm_with_tools:
        # Fallback
        if isinstance(last_message, ToolMessage):
            final = f"Meeting prep based on research:\n{last_message.content[:300]}"
            return {"messages": [AIMessage(final)], "final_answer": final,
                    "steps": [{"type": "reason", "content": "Generated meeting prep."}]}
        return {
            "messages": [AIMessage(content="", tool_calls=[{
                "name": "tavily_search", "args": {"query": "Langfuse Eval"}, "id": "call_meet_1"
            }])],
            "steps": [{"type": "reason", "content": "Searching for meeting context."}]
        }
    
    try:
        if isinstance(last_message, ToolMessage):
            clean = []
            tool_msgs, ai_msg, human_msg = [], None, None
            for msg in reversed(messages):
                if isinstance(msg, ToolMessage):
                    tool_msgs.insert(0, msg)
                elif isinstance(msg, AIMessage) and getattr(msg, 'tool_calls', None):
                    ai_msg = msg
                    break
                elif isinstance(msg, HumanMessage) and not human_msg:
                    human_msg = msg
            clean = ([human_msg] if human_msg else []) + ([ai_msg] if ai_msg else []) + tool_msgs
            if not clean:
                clean = messages[-3:]
            
            response = llm.invoke(clean)
            return {
                "messages": [response],
                "steps": [{"type": "reason", "content": "Synthesizing meeting prep."}],
                "final_answer": response.content
            }
        else:
            response = llm_with_tools.invoke(messages)
            if getattr(response, 'tool_calls', None):
                return {
                    "messages": [response],
                    "steps": [{"type": "reason",
                               "content": response.content or "Calling tools for meeting context."}]
                }
            return {
                "messages": [response],
                "steps": [{"type": "reason", "content": "Generated meeting prep."}],
                "final_answer": response.content
            }
    except Exception as e:
        err = f"Error: {e}"
        return {"messages": [AIMessage(err)], "final_answer": err}

def meeting_tool_node(state: MeetingState) -> dict:
    """Execute tools for meeting prep."""
    msgs = state.get("messages", [])
    last = msgs[-1] if msgs else None
    if not last or not getattr(last, 'tool_calls', None):
        return {}
    
    tool_map = {t.name: t for t in tools}
    tool_messages, tool_steps = [], []
    for tc in last.tool_calls:
        name, args, cid = tc["name"], tc["args"], tc["id"]
        if name in tool_map:
            try:
                result = tool_map[name].invoke(args)
                tool_messages.append(ToolMessage(content=str(result), tool_call_id=cid))
                tool_steps.append({"type": "act", "tool": name, "args": args, "result": str(result)[:200]})
            except Exception as e:
                tool_messages.append(ToolMessage(content=f"Error: {e}", tool_call_id=cid))
        else:
            tool_messages.append(ToolMessage(content=f"Unknown tool: {name}", tool_call_id=cid))
    return {"messages": tool_messages, "steps": tool_steps} if tool_messages else {}

def meeting_router(state: MeetingState) -> Literal["meeting_tools", "__end__"]:
    if len(state.get("steps", [])) >= MAX_STEPS:
        return "__end__"
    if state.get("final_answer"):
        return "__end__"
    msgs = state.get("messages", [])
    if msgs and isinstance(msgs[-1], AIMessage) and getattr(msgs[-1], 'tool_calls', None):
        return "meeting_tools"
    return "__end__"

# Build meeting prep graph
meeting_wf = StateGraph(MeetingState)
meeting_wf.add_node("meeting_reasoner", meeting_reasoner)
meeting_wf.add_node("meeting_tools", meeting_tool_node)
meeting_wf.add_edge(START, "meeting_reasoner")
meeting_wf.add_conditional_edges("meeting_reasoner", meeting_router)
meeting_wf.add_edge("meeting_tools", "meeting_reasoner")

meeting_memory = MemorySaver()
meeting_app = meeting_wf.compile(checkpointer=meeting_memory)

print("✅ Meeting Prep Agent compiled!")


In [ ]:
# Run the Meeting Prep Agent
print("=" * 60)
print("🏗️ MEETING PREP AGENT")
print("=" * 60)

meeting_config = {"configurable": {"thread_id": "meeting-prep-1"}}

meeting_state = {
    "messages": [HumanMessage(
        "Prep me for a meeting with Marc Kligen about Langfuse Eval. "
        "Marc Kligen is from Langfuse. "
        "Give me 5 agenda bullets, 3 questions to ask, and one demo idea."
    )],
    "steps": [],
    "final_answer": ""
}

result_meeting = meeting_app.invoke(meeting_state, config=meeting_config)

# Display the trace
print("\n📋 REACT TRACE:")
for i, step in enumerate(result_meeting.get("steps", []), 1):
    step_type = step.get("type", "unknown")
    if step_type == "reason":
        print(f"\n  Step {i} - 💭 REASON:")
        print(f"    {step.get('content', '')[:150]}")
    elif step_type == "act":
        print(f"\n  Step {i} - 🔧 ACT: {step.get('tool', '')}()")
        print(f"    📊 Result: {step.get('result', '')[:150]}...")

print("\n" + "=" * 60)
print("✅ MEETING PREP OUTPUT:")
print("=" * 60)
print(result_meeting.get("final_answer", "No answer generated"))


---

# 🎨 Interactive Streamlit UI

Let's build a beautiful Streamlit interface to interact with our ReAct agent! This UI demonstrates the agent in action with real-time responses.

**Features:**
- Interactive chat interface
- Real-time ReAct trace visualization
- Tool call visualization
- Memory management with thread IDs


In [ ]:
# Check if the Streamlit app file exists
import os

streamlit_path = os.path.join(os.path.dirname(os.path.abspath(".")), 
                               "multi-agent-systems", "week-2", "streamlit_app.py")

# Check in current directory first
if os.path.exists("streamlit_app.py"):
    streamlit_path = "streamlit_app.py"

if os.path.exists(streamlit_path):
    print("✅ Streamlit app file found: streamlit_app.py")
    print("\n📝 To run the Streamlit UI:")
    print("   1. Make sure all dependencies are installed:")
    print("      pip install -r requirements.txt")
    print("   2. Ensure your .env file has:")
    print("      - OPENAI_API_KEY=your_key")
    print("      - TAVILY_API_KEY=your_key (optional but recommended)")
    print("   3. Run the Streamlit app:")
    print("      streamlit run streamlit_app.py")
    print("\n💡 The UI will open in your browser at http://localhost:8501")
    print("\n🎨 Features:")
    print("   - Interactive chat interface")
    print("   - Real-time ReAct trace visualization")
    print("   - Tool call visualization")
    print("   - Memory management with thread IDs")
    print("   - Example queries in sidebar")
else:
    print("⚠️  streamlit_app.py not found in the current directory.")
    print("   It should be in the same directory as this notebook.")


### Running the Streamlit App

To run the interactive UI:

```bash
# Navigate to the week-2 directory
cd AI-Internship/multi-agent-systems/week-2

# Activate virtual environment (recommended)
source venv/bin/activate

# Install dependencies
pip install -r requirements.txt

# Run the app
streamlit run streamlit_app.py
```

The app will open at **http://localhost:8501** with:
- 💬 **Chat interface** for interacting with the ReAct agent
- 🔍 **ReAct trace** showing each Reason → Act → Observe step  
- 🔧 **Tool visualization** showing which tools were called
- 🧵 **Thread management** for testing memory/checkpointing


In [ ]:
# Preview the Streamlit app code
if os.path.exists("streamlit_app.py"):
    with open("streamlit_app.py", "r") as f:
        content = f.read()
    print(f"📄 streamlit_app.py ({len(content.splitlines())} lines)")
    print("─" * 40)
    # Show first 30 lines as preview
    for line in content.splitlines()[:30]:
        print(line)
    print("...")
    print(f"─" * 40)
    print(f"(Showing first 30 of {len(content.splitlines())} lines)")


---

# 📝 Summary

Congratulations! You've learned how to build stateful, intelligent agents with LangGraph.

## Key Takeaways

| Concept | What You Learned |
|---------|------------------|
| **Typed State** | Design state with TypedDict and reducers for controlled updates |
| **Reducers** | Use `operator.add` for append-only fields, no reducer for overwrite fields |
| **ReAct Pattern** | Reason → Act → Observe → Repeat loop that makes agents intelligent |
| **Conditional Edges** | Router functions that decide what happens next based on state |
| **Checkpointing** | Give agents memory using thread_id and checkpoint stores |
| **Termination** | Always add max-steps guards to prevent infinite loops |

## Patterns Covered

1. ✅ **Typed State Design** — Proper state schema with reducers
2. ✅ **ReAct Agent** — Full implementation with tools and routing
3. ✅ **Checkpointing** — Memory and persistence across runs
4. ✅ **Meeting Prep Agent** — Complete working example

## What's Next?

- **Complete the Challenge** — Extend the Meeting Prep Agent
- **Week 3**: Multi-Agent Systems — Subgraphs, supervisor pattern, shared vs scoped state
- **Week 4**: Production + Capstone — Deploy your multi-agent system

---

### 🔗 Resources

- [LangGraph Official Documentation](https://docs.langchain.com/oss/python/langgraph/overview) — Main overview and API reference
- [LangGraph Graph API](https://langchain-ai.github.io/langgraph/) — Detailed graph construction guide
- [LangGraph Persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/) — Checkpointing and memory guide
- [ReAct Paper](https://arxiv.org/abs/2210.03629) — Original ReAct research paper
- [LangSmith](https://smith.langchain.com) — Debug & trace your graphs

**Note:** This notebook uses the official LangGraph API. For checkpointing, we use `MemorySaver` which is the standard in-memory checkpoint store.

---

**Your Week 1 router + your Week 2 ReAct agent combine into a real multi-agent system in Week 3!** 🚀


# 🎉 End of Week 2 Notebook - Happy Learning!
